In [10]:
!pip install -r basic_rag_requirements.txt



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
## Retrieval augmented generation

import os
from dotenv import load_dotenv
load_dotenv()


True

In [2]:
os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")

In [3]:
from llama_index import VectorStoreIndex,SimpleDirectoryReader
documents=SimpleDirectoryReader("data").load_data()

In [4]:
documents

[Document(id_='37d2a352-1407-4885-8e15-8325be052814', embedding=None, metadata={'page_label': '1', 'file_name': 'standard-residential-lease-agreement-template.pdf', 'file_path': 'data\\standard-residential-lease-agreement-template.pdf', 'file_type': 'application/pdf', 'file_size': 144887, 'creation_date': '2025-03-03', 'last_modified_date': '2025-03-02', 'last_accessed_date': '2025-03-03'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='STANDARD LEASE AGREEMENT\n\xa0\n\xa0\nTHIS LEASE AGREEMENT hereinafter known as the "Lease" is made and entered into\n\xa0\nthis ____ day of _______________________,  20____, by and between the Landlord known\n\xa0\nas _______________________ with a mailing address _______________________, in the City\n\xa0\nof _________

In [5]:
index=VectorStoreIndex.from_documents(documents,show_progress=True)

c:\Users\amaan\Desktop\RAG Chatbot\basic_rag_venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating embeddings: 100%|██████████| 8/8 [00:01<00:00,  4.22it/s]


In [6]:
index

In [7]:
query_engine=index.as_query_engine()

In [8]:
from llama_index.retrievers import VectorIndexRetriever
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.indices.postprocessor import SimilarityPostprocessor

retriever=VectorIndexRetriever(index=index,similarity_top_k=4)
postprocessor=SimilarityPostprocessor(similarity_cutoff=0.80)

query_engine=RetrieverQueryEngine(retriever=retriever,
                                  node_postprocessors=[postprocessor])


In [9]:
import os.path
from llama_index import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    load_index_from_storage,
)

# check if storage already exists
PERSIST_DIR = "./storage"
if not os.path.exists(PERSIST_DIR):
    # load the documents and create the index
    documents = SimpleDirectoryReader("data").load_data()
    index = VectorStoreIndex.from_documents(documents)
    # store it for later
    index.storage_context.persist(persist_dir=PERSIST_DIR)
else:
    # load the existing index
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
    index = load_index_from_storage(storage_context)

# either way we can now query the index
query_engine = index.as_query_engine()
response = query_engine.query("What is the stance on retaliatory acts?")
print(response)

The Landlord is prohibited from making any type of retaliatory acts against the Tenant(s) including but not limited to restricting access to the Property, decreasing or canceling services or utilities, failure to repair appliances or fixtures, or any other type of activity that could be considered unjustified.
